# OWL_Demo1 Analyze Downloaded Data

# First things first (Install necessary packages)

### Install scapy for extracting data from `*.pcap` files

In [ ]:
!conda install -y --quiet -c conda-forge scapy

### Standard imports

In [ ]:
from pathlib import Path
import pandas as pd

#from mflib import owl_data
import owl_data  # For testing local file only

# CSV File path where parsed data are (will be) saved

In [ ]:
# Output file name
csv_path = './data.csv'

# (If CSV file not yet generated) Parse pcap files and save in one combined CSV file

You will need
- location of local `*.pcap` files. (Method below will look for pcap files recursively from the location.)
- output (csv) file name 

In [ ]:
root_data_dir = './'

List all the .pcap files under the root data dir (recursive search)

In [ ]:
pcap_files = owl_data.list_pcap_files(root_data_dir)
print(f"Found pcap files: {[str(p) for p in pcap_files]}")

### Extract entries from PCAP and add to the csv file

Args:

- `pcap_files`: list of pcap file paths
- `outfile (default="out.csv")`: path to output csv file
- `append_csv (default=False)`: if set to True, append to the existing csv file of the same name; if False, it will warn and exit in case there is already a csv file of the same name.
- `verbose (default=False)`: if True, it will print out the content of pcap files during the extraction process

Output CSV file should have 6 columns:
- source IP
- sent timestamp (epoch time)
- destination IP
- destinatin timestamp (epoch time)
- sequence number
- latency

In [ ]:
owl_data.convert_pcap_to_csv(pcap_files, outfile=csv_path)

# Analyze Latency Data 

## Create a pandas DataFrame from the csv file (incl. data cleansing)

In [ ]:
df = owl_data.convert_to_df(csv_path)
df.head()

## Example 1: Get latency data between two nodes

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
slice_name = "test-slice"

try:
    slice = fablib.get_slice(name=slice_name)  
except Exception as e:
    print(f"Exception: {e}")

node0 = slice.get_node(name="node0")
node1 = slice.get_node(name="node1")
node2 = slice.get_node(name="node2")

In [ ]:
owl_data.get_summary(df, node1, node2)

## Eample 2: Filter data when IP addresses are known

In [ ]:
src_ip = "10.131.132.2"
dst_ip = "10.143.4.2"

filtered_df = owl_data.filter_data(df, src_ip, dst_ip)
filtered_df.head()

## Example 3: Graph Latency data 

Requires additional package installation

In [ ]:
!conda install -y --quiet -c conda-forge -c plotly jupyter-dash

In [ ]:
owl_data.graph_latency_data(df, node0, node1)